In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API key
from config import gkey

In [2]:
# Import cities file into the cities_pd DataFrame
file_locater = "../Resources/Cities.csv"
df = pd.read_csv(file_locater)
df

,City,State
0,New York City,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania
...,...,...
95,Fremont,California
96,Baton Rouge,Louisiana
97,Richmond,Virginia
98,Boise,Idaho


In [3]:
# Add columns for lat, lng, airport name, airport address, airport rating.
# Hint: Be sure to specify "" as the initial value for each column.
df["lat"] = ""
df["lng"] = ""
df["airport name"] = ""
df["airport address"] = ""
df["airport rating"] = ""
df.head()

,City,State,lat,lng,airport name,airport address,airport rating
0,New York City,New York,,,,,
1,Los Angeles,California,,,,,
2,Chicago,Illinois,,,,,
3,Houston,Texas,,,,,
4,Philadelphia,Pennsylvania,,,,,


In [6]:
# This cell block is to grab a genereal idea of how to pull relevant information for further coding.

params = {}
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"

# get city and state from first row
city = df.loc[0, "City"]
state = df.loc[0, "State"]
target_city = f"({city},{state})"
#target_city = "df.loc[0,["City","State"]]"

# # add keyword to params dict
params['address'] = target_city
params['key'] = gkey

# # assemble url and make API request
# print(f"Retrieving Results for Index {index}: {target_city}.")
response = requests.get(base_url, params=params).json()
    
# # extract results
#results = response['results']
#pprint(response["results"][0]["geometry"]["location"])
lat = response["results"][0]["geometry"]["location"]["lat"]
lng = response["results"][0]["geometry"]["location"]["lng"]
print(f"lag:{lat}, and lng:{lng}")

lag:40.7127753, and lng:-74.0059728


In [7]:
# Loop through the cities_pd and get the lat/long for each city
# Hint: `requests.get(target_url).json()`
# Hint: `for index, row in cities_pd.iterrows():`
# Hint: `cities_pd.loc`

for index, row in df.iterrows():

    # get city and state from row
    city = row["City"]
    state = row["State"]
    target_city = f"({city}, {state})"

    # add address and API key to params dict
    params['address'] = target_city
    params["key"] = gkey

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {target_city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    lat = response["results"][0]["geometry"]["location"]["lat"]
    lng = response["results"][0]["geometry"]["location"]["lng"]
    
    
    try:
        print(f"geocode for {target_city} is ({lat},{lng}.)")
        
        df.loc[index, 'lat'] = lat
        df.loc[index, 'lng'] = lng

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: (New York City, New York).
geocode for (New York City, New York) is (40.7127753,-74.0059728.)
------------
Retrieving Results for Index 1: (Los Angeles, California).
geocode for (Los Angeles, California) is (34.0522342,-118.2436849.)
------------
Retrieving Results for Index 2: (Chicago, Illinois).
geocode for (Chicago, Illinois) is (41.8781136,-87.6297982.)
------------
Retrieving Results for Index 3: (Houston, Texas).
geocode for (Houston, Texas) is (29.7604267,-95.3698028.)
------------
Retrieving Results for Index 4: (Philadelphia, Pennsylvania).
geocode for (Philadelphia, Pennsylvania) is (39.9525839,-75.1652215.)
------------
Retrieving Results for Index 5: (Phoenix, Arizona).
geocode for (Phoenix, Arizona) is (33.4483771,-112.0740373.)
------------
Retrieving Results for Index 6: (San Antonio, Texas).
geocode for (San Antonio, Texas) is (29.4241219,-98.49362819999999.)
------------
Retrieving Results for Index 7: (San Diego, California).
geocode f

In [45]:
# This cell block is to grab a genereal idea of how to pull relevant information for further coding.

params = {}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# get lat, and lng from first row
lat = df.loc[2, "lat"]
lng = df.loc[2, "lng"]
location = f"{lat},{lng}"

# Parametere
params = {
    "location": location,  # location coords
    #"rankby": "distance",
    "radius": 50000,
    "keyword": "international+airport",
    "type": "airport",
    "key": gkey,
}

# # add keyword to params dict
#params['address'] = target_city
#params['key'] = gkey

# Assemble url and make API request
# print(f"Retrieving Results for Index {index}: {target_city}.")
response = requests.get(base_url, params=params).json()
#pprint(response)


# Extract results
name = response["results"][0]["name"]
rating = response["results"][0]["rating"]
address = response["results"][0]["vicinity"]


# lng = response["results"][0]["geometry"]["location"]["lng"]

print(f"Name of international airpor:{name}, and its rating is: {rating} and its address is: {address}")

Name of international airpor:O'Hare International Airport, and its rating is: 4 and its address is: 10000 W O'Hare Ave, Chicago


In [47]:
# Use the lat/lng we recovered to identify airports
# Hint: Code is similar to above block (i.e. you will need to use cities_pd.iterrows() again)
# Hint: Use the keyword search for "international+airport"
# Hint: Use a try-except to handle airports that are missing ratings

# Loop through the cities_pd and get the lat/long for each city
# Hint: `requests.get(target_url).json()`
# Hint: `for index, row in cities_pd.iterrows():`
# Hint: `cities_pd.loc`

for index, row in df.iterrows():

    # get lat and lng from each row
    lat = row["lat"]
    lng = row["lng"]
    location = f"{lat},{lng}"

    # add keyword to params dict
    params = {
        "location": location,  # index location coords
        #"rankby": "distance",
        "radius": 50000,
        "keyword": "international+airport",
        "key": gkey,
        "type": "airport",
    }

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()
    
    try:
        # extract results
        name = response["results"][0]["name"]
        rating = response["results"][0]["rating"]
        address = response["results"][0]["vicinity"]
    
    
    
        print(f"The nearest airport for this location ({location}) is {name}, and its rating is {rating}")
        
        df.loc[index, 'airport name'] = name
        df.loc[index, 'airport rating'] = rating
        df.loc[index, 'airport address'] = address

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: 40.7127753,-74.0059728.
The nearest airport for this location (40.7127753,-74.0059728) is Newark Liberty International Airport, and its rating is 3.7
------------
Retrieving Results for Index 1: 34.0522342,-118.2436849.
The nearest airport for this location (34.0522342,-118.2436849) is Los Angeles International Airport, and its rating is 3.9
------------
Retrieving Results for Index 2: 41.8781136,-87.6297982.
The nearest airport for this location (41.8781136,-87.6297982) is O'Hare International Airport, and its rating is 4
------------
Retrieving Results for Index 3: 29.7604267,-95.3698028.
The nearest airport for this location (29.7604267,-95.3698028) is George Bush Intercontinental Airport, and its rating is 4.2
------------
Retrieving Results for Index 4: 39.9525839,-75.1652215.
The nearest airport for this location (39.9525839,-75.1652215) is Philadelphia International Airport, and its rating is 3.8
------------
Retrieving Results for Index 5: 33.448

In [48]:
# Save Data to csv
save_to_location = "../Resources/Output.csv"
df.to_csv(save_to_location)

# Visualize to confirm airport data appears
df.head()

,City,State,lat,lng,airport name,airport address,airport rating
0,New York City,New York,40.712775,-74.005973,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.7
1,Los Angeles,California,34.052234,-118.243685,Los Angeles International Airport,"1 World Way, Los Angeles",3.9
2,Chicago,Illinois,41.878114,-87.629798,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",4
3,Houston,Texas,29.760427,-95.369803,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",4.2
4,Philadelphia,Pennsylvania,39.952584,-75.165222,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.8
